# Data preparation and analysis

## TEMPERATURA

In [67]:
import os
import sys
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import datetime

import pandas as pd
import numpy as np
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

from datetime import datetime
import numpy 

In [68]:
#Cargo el archivo con la informacion sobre el clima

In [69]:
folder = "/Users/mariabelenalberti/Downloads/smart-meters-in-london/"
filename = "weather_hourly_darksky.csv"
df_weather = pd.read_csv(folder + filename)

In [70]:
# Me quedo solo con la temperatura y las fechas

In [71]:
df_weather = df_weather[['temperature', 'time']]
df_weather.columns = ['temperature', 'DateTime']
df_weather['DateTime'] = pd.to_datetime(df_weather['DateTime'])
df_weather['year'] = df_weather['DateTime'].dt.year
df_weather['month'] = df_weather['DateTime'].dt.month
df_weather['day'] = df_weather['DateTime'].dt.day

In [72]:
df_weather.head()

,temperature,DateTime,year,month,day
0,10.24,2011-11-11 00:00:00,2011,11,11
1,9.76,2011-11-11 01:00:00,2011,11,11
2,9.46,2011-11-11 02:00:00,2011,11,11
3,9.23,2011-11-11 03:00:00,2011,11,11
4,9.26,2011-11-11 04:00:00,2011,11,11


In [73]:
# Porque solo tenemos los años 2012 y 2013 completos, descarto 2011 y 2014

In [74]:
#Filtro el año 2012 y 2013
df_weather_2012 = df_weather[df_weather.year==2012][['temperature', 'month']]
df_weather_2013= df_weather[df_weather.year==2013][['temperature', 'month']]

# Concatenos los dos df
df_weather = pd.concat([df_weather_2012, df_weather_2013],keys=['2012','2013'],names=['year'])

# Agrupo el df por año y mes y me quedo con el promedio de temperatura en el mes
df_weather = df_weather.groupby(by=['year','month'])['temperature'].mean().to_frame()
df_weather.reset_index(level=0, inplace=True)

In [75]:
# Dataframe final con la temperatura de 2012 y 2013 con los promedios de temperatura por mes
df_weather

,year,temperature
month,,
1,2012,6.776707
2,2012,4.868534
3,2012,9.307433
4,2012,8.600569
5,2012,13.492755
6,2012,14.950514
7,2012,16.838038
8,2012,18.345605
9,2012,14.628125


## CONSUMO + HOUSEHOLD ACORN

In [76]:
# Consumo

folder = "/Users/mariabelenalberti/Downloads/smart-meters-in-london/"
dataset_folder="daily_dataset/"

import glob
import pandas as pd

path = "/Users/mariabelenalberti/Downloads/smart-meters-in-london/daily_dataset"
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# Concateno todos los bloques de datos
daily_complete = pd.concat(dfs, ignore_index=True)
pd.to_datetime(daily_complete.day)

0         2011-12-07
1         2011-12-08
2         2011-12-09
3         2011-12-10
4         2011-12-11
5         2011-12-12
6         2011-12-13
7         2011-12-14
8         2011-12-15
9         2011-12-16
10        2011-12-17
11        2011-12-18
12        2011-12-19
13        2011-12-20
14        2011-12-21
15        2011-12-22
16        2011-12-23
17        2011-12-24
18        2011-12-25
19        2011-12-26
20        2011-12-27
21        2011-12-28
22        2011-12-29
23        2011-12-30
24        2011-12-31
25        2012-01-01
26        2012-01-02
27        2012-01-03
28        2012-01-04
29        2012-01-05
             ...    
3510403   2013-04-16
3510404   2013-04-17
3510405   2013-04-18
3510406   2013-04-19
3510407   2013-04-20
3510408   2013-04-21
3510409   2013-04-22
3510410   2013-04-23
3510411   2013-04-24
3510412   2013-04-25
3510413   2013-04-26
3510414   2013-04-27
3510415   2013-04-28
3510416   2013-04-29
3510417   2013-04-30
3510418   2013-05-01
3510419   201

In [77]:
# Household Acorn

folder = "/Users/mariabelenalberti/Downloads/smart-meters-in-london/"
filename = "informations_households.csv"
households = pd.read_csv(folder + filename)

In [81]:
households["Acorn"].value_counts()

ACORN-E    1567
ACORN-Q     831
ACORN-F     684
ACORN-H     455
ACORN-L     342
ACORN-D     292
ACORN-G     205
ACORN-K     165
ACORN-A     157
ACORN-N     152
ACORN-C     151
ACORN-M     113
ACORN-J     112
ACORN-P     110
ACORN-O     103
ACORN-I      51
ACORN-U      49
ACORN-B      25
ACORN-        2
Name: Acorn, dtype: int64

In [33]:
# Merge de ambas df usando key = LCLid

energy_dataframe = pd.merge(daily_complete, households, on='LCLid')

#Vemos como se cargaron las cosas
energy_dataframe.info()

# Hay que transformar a la variable fecha para poder utilizarla
energy_dataframe['day']= pd.to_datetime(energy_dataframe['day'],format = '%Y-%m-%d')
energy_dataframe['year'] = pd.DatetimeIndex(energy_dataframe['day']).year
energy_dataframe['month'] = pd.DatetimeIndex(energy_dataframe['day']).month

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3510433 entries, 0 to 3510432
Data columns (total 13 columns):
LCLid            object
day              object
energy_median    float64
energy_mean      float64
energy_max       float64
energy_count     int64
energy_std       float64
energy_sum       float64
energy_min       float64
stdorToU         object
Acorn            object
Acorn_grouped    object
file             object
dtypes: float64(6), int64(1), object(6)
memory usage: 375.0+ MB


In [34]:
# Otra vez, para poder tener un análisis consitente, nos quedamos con los años 2012 y 2013

energy_dataframe_2012 = energy_dataframe[energy_dataframe.year==2012][['LCLid','Acorn_grouped', 'energy_sum','month','year']]
energy_dataframe_2013 = energy_dataframe[energy_dataframe.year==2013][['LCLid','Acorn_grouped', 'energy_sum','month','year']]

# Concateno las df resutlantes
df_energy = pd.concat([energy_dataframe_2012,energy_dataframe_2013],keys=['2012','2013'],names=['year'],ignore_index=True)

In [35]:
df_energy.head()

,LCLid,Acorn_grouped,energy_sum,month,year
0,MAC000027,Comfortable,8.962,1,2012
1,MAC000027,Comfortable,7.159,1,2012
2,MAC000027,Comfortable,6.430,1,2012
3,MAC000027,Comfortable,5.556,1,2012
4,MAC000027,Comfortable,5.420,1,2012


In [36]:
#Agrupo por acorn, año y mes, quedandome con el promedio de la suma de consumo
df_energy = df_energy.groupby(by=['Acorn_grouped','year','month'],sort =False,as_index=False)['energy_sum'].mean()


In [37]:
df_energy.head()

,Acorn_grouped,year,month,energy_sum
0,Comfortable,2012,1,11.207878
1,Comfortable,2012,2,12.169113
2,Comfortable,2012,3,10.044413
3,Comfortable,2012,4,9.404593
4,Comfortable,2012,5,8.498311


## CONSUMO + ACORN + TEMPERATURA

In [38]:
#df_weather
#df_energy

#tengo el consumo por mes y año de cada acorn
#la temperatura por mes por año
#no tengo idea como merge eso 

In [ ]:
# 2012

In [41]:
#Temperatura 2012

In [43]:
df_weather_2012 = df_weather_2012.groupby(by=['month'])['temperature'].mean().to_frame()
df_weather_2012.reset_index(level=0, inplace=True)

In [44]:
df_weather_2012

,month,temperature
0,1,6.776707
1,2,4.868534
2,3,9.307433
3,4,8.600569
4,5,13.492755
5,6,14.950514
6,7,16.838038
7,8,18.345605
8,9,14.628125
9,10,10.960981


In [ ]:
# Energia 2012

In [45]:
energy_dataframe_2012 = energy_dataframe_2012.groupby(by=['Acorn_grouped','month'],sort =False,as_index=False)['energy_sum'].mean()
energy_dataframe_2012.reset_index(level=0, inplace=True)

In [47]:
year_2012 = pd.merge(df_weather_2012, energy_dataframe_2012, on='month')

In [55]:
year_2012

,month,temperature,index,Acorn_grouped,energy_sum
0,1,6.776707,0,Comfortable,11.207878
1,1,6.776707,12,Adversity,12.204682
2,1,6.776707,35,Affluent,14.429848
3,1,6.776707,45,ACORN-U,12.439628
4,2,4.868534,1,Comfortable,12.169113
5,2,4.868534,13,Adversity,11.772610
6,2,4.868534,34,Affluent,14.691953
7,2,4.868534,46,ACORN-U,12.128813
8,3,9.307433,2,Comfortable,10.044413
9,3,9.307433,14,Adversity,9.842762


In [ ]:
#2013

In [49]:
#Temperatura 2013

In [50]:
df_weather_2013 = df_weather_2013.groupby(by=['month'])['temperature'].mean().to_frame()
df_weather_2013.reset_index(level=0, inplace=True)

In [51]:
# Energia 2012

In [52]:
energy_dataframe_2012 = energy_dataframe_2012.groupby(by=['Acorn_grouped','month'],sort =False,as_index=False)['energy_sum'].mean()
energy_dataframe_2012.reset_index(level=0, inplace=True)

In [53]:
year_2013 = pd.merge(df_weather_2012, energy_dataframe_2012, on='month')

In [ ]:
# Concatenate 2012 y 2013

In [63]:
year_2012['year'] = str(2012)
year_2013['year'] = str(2013)

In [64]:
df = pd.concat([year_2012,year_2013],keys=['2012','2013'],names=['year'],ignore_index=True)

In [65]:
df

,month,temperature,index,Acorn_grouped,energy_sum,year
0,1,6.776707,0,Comfortable,11.207878,2012
1,1,6.776707,12,Adversity,12.204682,2012
2,1,6.776707,35,Affluent,14.429848,2012
3,1,6.776707,45,ACORN-U,12.439628,2012
4,2,4.868534,1,Comfortable,12.169113,2012
5,2,4.868534,13,Adversity,11.772610,2012
6,2,4.868534,34,Affluent,14.691953,2012
7,2,4.868534,46,ACORN-U,12.128813,2012
8,3,9.307433,2,Comfortable,10.044413,2012
9,3,9.307433,14,Adversity,9.842762,2012


In [66]:
df.to_csv("df.csv", index=False, sep=',', encoding='utf-8')